## Gradio Demo app

The following notebook showcases an internal decision making tool used to compare the performance of different models.

> **NOTE:** This notebook was run on Google Colab with the GPU Accelator Enabled. The data used for this notebook is available on the MDS-Capstone-GDLR OneDrive. You can find the models used in this notebook by downloading the trained_models sub-directory.  

### Imports

In [1]:
import gradio as gr # pip install gradio==1.7.6
import torch
from torchvision import transforms
import requests
from PIL import Image

import numpy as np
import pandas as pd

from torchvision import transforms, models, datasets, utils
from collections import OrderedDict
from torch import nn, optim
import os

In [2]:
os.path.dirname(os.getcwd()) 

'/Users/user/mds/capstone/capstone-gdrl-lipo'

### Setup Models for Prediction

In [3]:
MODEL_DIR = os.path.dirname(os.getcwd()) + '/trained_models/' # download trained models from OneDrive 

In [4]:
# set up the model layers

# densenet
dense_model = models.densenet121(pretrained=True)
new_layers = nn.Sequential(
    OrderedDict([
                 ("new1", nn.Linear(1024, 500)),
                 ("relu", nn.ReLU()),
                 ("new2", nn.Linear(500, 1)),
                 ]))
dense_model.classifier = new_layers

# inception 
inception_model = models.inception_v3(pretrained=True)
inception_model.fc = nn.Linear(2048, 1)
inception_model.aux_logits = False

# vgg16
vgg_model = models.vgg16_bn(pretrained=True)
num_features = vgg_model.classifier[6].in_features
features = list(vgg_model.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, 1)]) # Add our layer with 4 outputs
vgg_model.classifier = nn.Sequential(*features) # Replace the model classifier

# resnet 
resnet_model = models.resnet50(pretrained=True)
resnet_model.fc = nn.Sequential(nn.Linear(2048, 512),
                             nn.ReLU(),
                             nn.Linear(512, 1))

In [5]:
# define a function that sets up the app 
def state_dict_loader(model, model_path):
    '''Loads the model given the predefined layers, path of the trained model and whether a GPU is present or not
    '''
    model_path = str(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if device.type == 'cuda':
        model.load_state_dict(torch.load(MODEL_DIR + model_path))
    else:
        model.load_state_dict(torch.load(MODEL_DIR + model_path, 
                                       map_location=torch.device('cpu')))
    model.eval()

In [7]:
# load saved models 
state_dict_loader(dense_model, 'trained_models_June2/densenet_final.pth')
state_dict_loader(inception_model, 'trained_models_May25/inception_bo_simple.pth')
state_dict_loader(vgg_model, 'trained_models_June2/vgg16-final.pth')
state_dict_loader(resnet_model, 'trained_models_May25/resnet.pt')

# define functions that take an image input for gradio
def classify_densenet(inp):
    inp = Image.fromarray(inp.astype('uint8'), 'RGB')
    image_tensor = transforms.functional.to_tensor(inp.resize((300, 300))).unsqueeze(0)
    pos = float(torch.sigmoid(dense_model(image_tensor)).detach().numpy())
    neg = 1 - pos
    return {'Negative': neg, 'Positive': pos}

# uncomment to look at all the models 
def classify_inception(inp):
    inp = Image.fromarray(inp.astype('uint8'), 'RGB')
    image_tensor = transforms.functional.to_tensor(inp.resize((300, 300))).unsqueeze(0)
    pos = float(torch.sigmoid(inception_model(image_tensor)).detach().numpy())
    neg = 1 - pos
    return {'Negative': neg, 'Positive': pos}

def classify_vgg(inp):
    inp = Image.fromarray(inp.astype('uint8'), 'RGB')
    image_tensor = transforms.functional.to_tensor(inp.resize((300, 300))).unsqueeze(0)
    pos = float(torch.sigmoid(vgg_model(image_tensor)).detach().numpy())
    neg = 1 - pos
    return {'Negative': neg, 'Positive': pos}

def classify_resnet(inp):
    inp = Image.fromarray(inp.astype('uint8'), 'RGB')
    image_tensor = transforms.functional.to_tensor(inp.resize((300, 300))).unsqueeze(0)
    pos = float(torch.sigmoid(resnet_model(image_tensor)).detach().numpy())
    neg = 1 - pos
    return {'Negative': neg, 'Positive': pos}

### Pass the functions to Gradio

In [ ]:
# launch gradio app 
inputs = gr.inputs.Image()
outputs = gr.outputs.Label()
gr.Interface(fn=[classify_inception, classify_vgg, classify_densenet, classify_resnet], # uncomment to launch all models
             #fn = classify_densenet,
             inputs=inputs, outputs=outputs, interpretation="default").launch(debug=True)

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...
